In [5]:
#q8
!hdfs dfsadmin -fs hdfs://boss:9000 -report

# import urllib.request

# url = "https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv"
# filename = "hdma-wi-2021.csv"

# urllib.request.urlretrieve(url, filename)

# # # Remove the files if they already exist in HDFS
# # !hdfs dfs -rm -f hdfs://boss:9000/single.csv
# # !hdfs dfs -rm -f hdfs://boss:9000/double.csv

# # !HADOOP_user=hdfs hdfs dfs -D dfs.replication=1 -put hdma-wi-2021.csv hdfs://boss:9000/single.csv
# # !HADOOP_user=hdfs hdfs dfs -D dfs.replication=2 -put hdma-wi-2021.csv hdfs://boss:9000/double.csv

Configured Capacity: 25821052928 (24.05 GB)
Present Capacity: 18680159068 (17.40 GB)
DFS Remaining: 18416279552 (17.15 GB)
DFS Used: 263879516 (251.66 MB)
DFS Used%: 1.41%
Replicated Blocks:
	Under replicated blocks: 167
	Blocks with corrupt replicas: 0
	Missing blocks: 84
	Missing blocks (with replication factor 1): 84
	Low redundancy blocks with highest priority to recover: 167
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 172.18.0.2:9866 (project-4-junho_4-dn-1.project-4-junho_4_default)
Hostname: ad2b6cf061c0
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 263879516 (251.66 MB)
Non DFS Used: 7124116644 (6.63 GB)
DFS Remaining: 18416279552 (17.15 GB)
DFS Used%: 1.02%
DFS Rem

In [2]:
import time
time.sleep(30)

In [6]:
#q9
import requests
import json

# WebHDFS URL for the file
# url = "http://boss:9870/webhdfs/v1/single.csv?op=GETFILEBLOCKLOCATIONS&offset=0&length=0"
url = "http://boss:9870/webhdfs/v1/single.csv?op=GETFILEBLOCKLOCATIONS"
# Send a GET request
response = requests.get(url)

# Convert the response to JSON
data = response.json()
data

redirect_url = data['BlockLocations']['BlockLocation']

#block_distribution = {}

#redirect_url
#Initialize an empty dictionary to store the block distribution
block_distribution = {}
data['BlockLocations']['BlockLocation']
# Loop through each block
for block in data['BlockLocations']['BlockLocation']:
    # Extract the container ID from the hosts
    if(block['hosts'] != []):
        container_id = block['hosts'][0]
    else:
        container_id = 'lost'
    
    if container_id in block_distribution:
        block_distribution[container_id] += 1
    else:
        block_distribution[container_id] = 1

# # Print the block distribution
block_distribution

{'lost': 84, 'ad2b6cf061c0': 83}

In [7]:
#q10
import pyarrow as pa
import pyarrow.fs

# Connect to HDFS
hdfs = pa.fs.HadoopFileSystem("boss", 9000)

url = "http://boss:9870/webhdfs/v1/single.csv?op=GETFILEBLOCKLOCATIONS"

# Send a GET request
response = requests.get(url)

# Convert the response to JSON
data = response.json()

# Initialize the count of "Single Family"
count = 0

# Loop through each block
for block in data['BlockLocations']['BlockLocation']:
    # Extract the container ID from the hosts
    if block["hosts"] != []:
        # Open the file
        with hdfs.open_input_file("hdfs://boss:9000/single.csv") as file:
            # Read the first 10 bytes of the file
            bytes = file.read_at(block["length"], block["offset"])
            count += str(bytes).count("Single Family")

count

221298